                    GNU GENERAL PUBLIC LICENSE
                       Version 3, 29 June 2007

Ejecutar secuencialmente cada celda del notebook, cambiando segun corresponda los nombres de archivo.

In [1]:
from Bio import SeqIO
# para leer fasta
# si da error "No module named Bio", 
# install Biopython, ej: pip install Biopython o sudo apt-get install python-biopython

In [3]:
#lectura de secuencias objetivo
records=list(SeqIO.parse("DQB 9F.fa", "fasta"))

#Aqui deberia prever la lectura de los leidos del cromatograma en R

In [4]:
records[0]

SeqRecord(seq=Seq('GATTTCGTGTACCAGTTTAAGTTCGAGTGCTATTTCACCAACGGGACGGGAGCG...GGT', SingleLetterAlphabet()), id='DQB9F', name='DQB9F', description='DQB9F', dbxrefs=[])

In [5]:
#convierto las secuencias a strings y dscripciones
seq=[str(r.seq) for r in records]
desc=[str(r.description).strip() for r in records]
print desc[1]
print seq[1]
#convierto las secuencias en strings - OJO QUE DEJO EL ID


IndexError: list index out of range

In [6]:
def seq_a_lista(s):
    #recibe un string y devuelve una lista
    # proceso las [A/G] eliminando los [] y uniendo los dos valores en torno a / como listas
    # TODO: ver que pasa con los [./.]. De movida debe quedar .. ya que no estoy usando sets, que eliminarian la duplicación
    s=s.replace('[','').replace(']','')
    sl= [x for x in s]
    for i,x in enumerate(sl):
        if x=='/':
            sl[i-1]+=sl[i+1]
            del sl[i:i+2]
    return sl

def lista_a_seq(l):
    '''recibe una lista y devuelve un string con corchetes y barra en los polimorfismos
    '''
    ll=[x if len(x)<2 else '[' + x[0] + '/' + x[1] + ']' for x in l] #'/'.join(x)
    return ''.join(ll)
    
def seq_a_string(s):
    '''recibe un string con corchetes, lo transforma en lista y lo traduce a IUPAC
    '''
    return lista_a_iupac(seq_a_lista(s))

def guion_a_vacio(l): # creo que ya no se usa, ver. recibe un string o una lista y convierte guiones en vacíos
    return [ x if (x<>'-')  else '' for x in la]

def vacio_a_guion(l):
    '''recibe una lista y convierte los vacíos en guiones -
       si recibe un string lo devuelve tal cual
    '''
    return [ x if x else '-' for x in l]

def unir_haplotipos(la,lb):
    '''recibe dos listas de haplotipos de referencia
       maneja secuencias con guiones y las convierte a listas con vacíos
       devuelve una lista de la union de las dos listas elemento a elemento como conjunto (sin duplicados)
    '''
    la=[ x if (x<>'-')  else '' for x in la]
    lb=[ x if (x<>'-')  else '' for x in lb]
    return [''.join(list(set(la[i]+lb[i]))) for i in range(len(la))]

def ref_haplo(la):
    '''recibe un haplotipo de referencia, con los guiones convertidos en vacíos
       completa los guiones - con los datos del haplotipo de referencia
       
    '''
    return [ refseq[i] if not x or x=='-'  else x for i,x in enumerate(la)]
    #return [ x if (x in 'ACGT')  else refseq[i] for i,x in enumerate(la)] ## no son equivalentes, y este funciona siempre? Ver   

In [7]:
print len(seq)

1


In [8]:
for s in seq: #imprimo separado por espacios para ver
    print ' '.join(seq_a_lista(s))

G A T T T C G T G T A C C A G T T T A A G T T C G A G T G C T A T T T C A C C A A C G G G A C G G G A G C G G G T G C G G C T T C T G A C T A A A T A C A T C T A T A A C C G G G A G G A G T T C G T G C G C T T C G A C A G C G A C G T G G G G G A G T A C C G G G C G G T C A C G G A G C T C G G G C G G C C C G A C G C T G A G T A C T G G A A C C C G C A G A A G G A C G A G A T G G A C C G G G T A C G G G C C G A G C T G G A C A C G G T G T G C A G A C A C A A C T A C G G G G T G G A A G A G C T C A C C A C G T T G C A G C G G C G A G G T


In [10]:
#lectura del alineamiento
f=open('alineamiento DQB1.txt')
lineas=f.readlines()

# Queda en el diccionario d hasheado por id
d={}
ref=''
for i,l in enumerate(lineas):
    if (len(l)>20 and l[0]==' ' and l[1]!=' '): # uso un patron para identificar las lineas con datos
        id= l[:20].strip() #las corto por ancho fijo del id
        #print i, id
        sseq= l[21:].replace(' ','').rstrip() # elimino espacis
        #print id
        #print seq
        if id in d.keys(): # y las guardo (concatenadas) en un diccionario indexado por id
            d[id]+=sseq
        else:
            d[id]=sseq
        if not ref: ref=id # me estoy quedando con la primera secuencia asumiendo que es la de referencia
        d[id]=d[id].replace('*','-')
f.close()


In [11]:
# no hace falta, pero pongo la secuencia de referencia en esta variable (despues reorganizo) ## La uso??
ref
refseq = d[ref]
print ref, refseq

DLA-DQB1*00101 GATTTCGTGTACCAGTTTAAGGGCGAGTGCTATTTCACCAACGGGACGGAGCGGGTGCGGCTTCTGACGAGAGACATCTATAACCGGGAGGAGCACGTGCGCTTCGACAGCGACGTGGGGGAGTACCGGGCGGTCACGGAGCTCGGGCGGCCGGACGCTGAGTACTGGAACGGGCAGAAGGAGCTCTTGGAGCGGAGGCGGGCCGAGGTGGACACGGTGTGCAGACACAACTACGGGAGGGAAGAGCTCACCACGTTGCAGCGGCGA


In [12]:
seqs=[]
for s in seq:
    seqs.append(seq_a_lista(s))
print seqs

[['G', 'A', 'T', 'T', 'T', 'C', 'G', 'T', 'G', 'T', 'A', 'C', 'C', 'A', 'G', 'T', 'T', 'T', 'A', 'A', 'G', 'T', 'T', 'C', 'G', 'A', 'G', 'T', 'G', 'C', 'T', 'A', 'T', 'T', 'T', 'C', 'A', 'C', 'C', 'A', 'A', 'C', 'G', 'G', 'G', 'A', 'C', 'G', 'G', 'G', 'A', 'G', 'C', 'G', 'G', 'G', 'T', 'G', 'C', 'G', 'G', 'C', 'T', 'T', 'C', 'T', 'G', 'A', 'C', 'T', 'A', 'A', 'A', 'T', 'A', 'C', 'A', 'T', 'C', 'T', 'A', 'T', 'A', 'A', 'C', 'C', 'G', 'G', 'G', 'A', 'G', 'G', 'A', 'G', 'T', 'T', 'C', 'G', 'T', 'G', 'C', 'G', 'C', 'T', 'T', 'C', 'G', 'A', 'C', 'A', 'G', 'C', 'G', 'A', 'C', 'G', 'T', 'G', 'G', 'G', 'G', 'G', 'A', 'G', 'T', 'A', 'C', 'C', 'G', 'G', 'G', 'C', 'G', 'G', 'T', 'C', 'A', 'C', 'G', 'G', 'A', 'G', 'C', 'T', 'C', 'G', 'G', 'G', 'C', 'G', 'G', 'C', 'C', 'C', 'G', 'A', 'C', 'G', 'C', 'T', 'G', 'A', 'G', 'T', 'A', 'C', 'T', 'G', 'G', 'A', 'A', 'C', 'C', 'C', 'G', 'C', 'A', 'G', 'A', 'A', 'G', 'G', 'A', 'C', 'G', 'A', 'G', 'A', 'T', 'G', 'G', 'A', 'C', 'C', 'G', 'G', 'G', 'T', 'A', 'C'

In [13]:
print seqs[0]

['G', 'A', 'T', 'T', 'T', 'C', 'G', 'T', 'G', 'T', 'A', 'C', 'C', 'A', 'G', 'T', 'T', 'T', 'A', 'A', 'G', 'T', 'T', 'C', 'G', 'A', 'G', 'T', 'G', 'C', 'T', 'A', 'T', 'T', 'T', 'C', 'A', 'C', 'C', 'A', 'A', 'C', 'G', 'G', 'G', 'A', 'C', 'G', 'G', 'G', 'A', 'G', 'C', 'G', 'G', 'G', 'T', 'G', 'C', 'G', 'G', 'C', 'T', 'T', 'C', 'T', 'G', 'A', 'C', 'T', 'A', 'A', 'A', 'T', 'A', 'C', 'A', 'T', 'C', 'T', 'A', 'T', 'A', 'A', 'C', 'C', 'G', 'G', 'G', 'A', 'G', 'G', 'A', 'G', 'T', 'T', 'C', 'G', 'T', 'G', 'C', 'G', 'C', 'T', 'T', 'C', 'G', 'A', 'C', 'A', 'G', 'C', 'G', 'A', 'C', 'G', 'T', 'G', 'G', 'G', 'G', 'G', 'A', 'G', 'T', 'A', 'C', 'C', 'G', 'G', 'G', 'C', 'G', 'G', 'T', 'C', 'A', 'C', 'G', 'G', 'A', 'G', 'C', 'T', 'C', 'G', 'G', 'G', 'C', 'G', 'G', 'C', 'C', 'C', 'G', 'A', 'C', 'G', 'C', 'T', 'G', 'A', 'G', 'T', 'A', 'C', 'T', 'G', 'G', 'A', 'A', 'C', 'C', 'C', 'G', 'C', 'A', 'G', 'A', 'A', 'G', 'G', 'A', 'C', 'G', 'A', 'G', 'A', 'T', 'G', 'G', 'A', 'C', 'C', 'G', 'G', 'G', 'T', 'A', 'C',

In [14]:
print len(d), d.keys()

36 ['DLA-DQB1*01701', 'DLA-DQB1*01401', 'DLA-DQB1*02201', 'DLA-DQB1*008012', 'DLA-DQB1*01501', 'DLA-DQB1*02801', 'DLA-DQB1*01201', 'DLA-DQB1*00101', 'DLA-DQB1*02401', 'DLA-DQB1*03001', 'DLA-DQB1*02601', 'DLA-DQB1*02101', 'DLA-DQB1*01801', 'DLA-DQB1*01901', 'DLA-DQB1*02701', 'DLA-DQB1*00802', 'DLA-DQB1*03101', 'DLA-DQB1*01301', 'DLA-DQB1*01302', 'DLA-DQB1*01303', 'DLA-DQB1*03201', 'DLA-DQB1*00301', 'DLA-DQB1*01101', 'DLA-DQB1*02301', 'DLA-DQB1*00201', 'DLA-DQB1*03401', 'DLA-DQB1*00501', 'DLA-DQB1*00502', 'DLA-DQB1*03501', 'DLA-DQB1*00401', 'DLA-DQB1*008011', 'DLA-DQB1*00701', 'DLA-DQB1*01601', 'DLA-DQB1*03301', 'DLA-DQB1*02002', 'DLA-DQB1*02001']


In [15]:
#Generación de todos los haplotipos posibles
haplos=[]
hh=d.keys()
f=open('haplos.log','w')

def log(l): #aca defino la funcion para generar los logs
    for x in l:
        f.write(str(x))
    f.write('\n')
    
for x in hh:
    for y in hh:
        log((x,y))#print x,y
        log(('x :', d[x]))#print 'x:', d[x]
        log(('y :', d[y]))#print 'y:', d[y]
        log(('xr:',ref_haplo(d[x])))
        log(('yr:',ref_haplo(d[y])))
        dd= unir_haplotipos(ref_haplo(d[x]), ref_haplo(d[y]))
        log(('z0:', lista_a_seq((dd))))
        log(('z1:', lista_a_seq(vacio_a_guion(dd))))
        log(('z2:', lista_a_seq(dd)))
        log(('z3:', dd))
        haplos.append([[x,y],dd])

f.close()

- Leer archivo(s) de lecturas
- Leer archivo(s) de haplotipos
- Generar archivo de ranking
    En el original:
        * lectura, par de haplos, ranking, diferencias


In [16]:
print len(haplos), len(hh), len(hh)*len(hh)
print hh[11], haplos[11]

1296 36 1296
DLA-DQB1*02101 [['DLA-DQB1*01701', 'DLA-DQB1*02101'], ['G', 'A', 'T', 'T', 'T', 'C', 'G', 'T', 'G', 'T', 'AT', 'C', 'C', 'A', 'G', 'T', 'TG', 'T', 'A', 'A', 'G', 'G', 'C', 'C', 'G', 'A', 'G', 'T', 'G', 'C', 'T', 'A', 'T', 'T', 'T', 'C', 'A', 'C', 'C', 'A', 'A', 'C', 'G', 'G', 'G', 'A', 'C', 'G', 'G', 'A', 'G', 'C', 'G', 'G', 'G', 'T', 'G', 'C', 'G', 'G', 'CT', 'T', 'T', 'C', 'T', 'G', 'AG', 'C', 'TG', 'A', 'AG', 'A', 'AT', 'AG', 'C', 'A', 'T', 'C', 'T', 'A', 'T', 'A', 'A', 'C', 'C', 'G', 'G', 'G', 'A', 'G', 'G', 'A', 'G', 'T', 'T', 'C', 'G', 'T', 'G', 'C', 'G', 'C', 'T', 'T', 'C', 'G', 'A', 'C', 'A', 'G', 'C', 'G', 'A', 'C', 'G', 'T', 'G', 'G', 'G', 'G', 'G', 'A', 'G', 'T', 'AT', 'C', 'C', 'G', 'G', 'G', 'C', 'G', 'G', 'T', 'C', 'A', 'C', 'G', 'G', 'A', 'G', 'C', 'T', 'C', 'G', 'G', 'G', 'C', 'G', 'G', 'C', 'C', 'C', 'G', 'AT', 'C', 'G', 'C', 'T', 'G', 'A', 'G', 'T', 'AC', 'C', 'T', 'G', 'G', 'A', 'A', 'C', 'G', 'G', 'G', 'C', 'A', 'G', 'A', 'A', 'G', 'G', 'A', 'G', 'AT', 

In [17]:
print len(seqs)

1


In [18]:
for ss in seqs:
    print lista_a_seq(ss)

GATTTCGTGTACCAGTTTAAGTTCGAGTGCTATTTCACCAACGGGACGGGAGCGGGTGCGGCTTCTGACTAAATACATCTATAACCGGGAGGAGTTCGTGCGCTTCGACAGCGACGTGGGGGAGTACCGGGCGGTCACGGAGCTCGGGCGGCCCGACGCTGAGTACTGGAACCCGCAGAAGGACGAGATGGACCGGGTACGGGCCGAGCTGGACACGGTGTGCAGACACAACTACGGGGTGGAAGAGCTCACCACGTTGCAGCGGCGAGGT


In [19]:
s=seqs[0]
print s
print ':', lista_a_seq(s)
#print s, len(s)
print 'haplos:', haplos[4]
h=ref_haplo(haplos[4][1])
print 'h:',lista_a_seq(h)
#print h, len(h)

['G', 'A', 'T', 'T', 'T', 'C', 'G', 'T', 'G', 'T', 'A', 'C', 'C', 'A', 'G', 'T', 'T', 'T', 'A', 'A', 'G', 'T', 'T', 'C', 'G', 'A', 'G', 'T', 'G', 'C', 'T', 'A', 'T', 'T', 'T', 'C', 'A', 'C', 'C', 'A', 'A', 'C', 'G', 'G', 'G', 'A', 'C', 'G', 'G', 'G', 'A', 'G', 'C', 'G', 'G', 'G', 'T', 'G', 'C', 'G', 'G', 'C', 'T', 'T', 'C', 'T', 'G', 'A', 'C', 'T', 'A', 'A', 'A', 'T', 'A', 'C', 'A', 'T', 'C', 'T', 'A', 'T', 'A', 'A', 'C', 'C', 'G', 'G', 'G', 'A', 'G', 'G', 'A', 'G', 'T', 'T', 'C', 'G', 'T', 'G', 'C', 'G', 'C', 'T', 'T', 'C', 'G', 'A', 'C', 'A', 'G', 'C', 'G', 'A', 'C', 'G', 'T', 'G', 'G', 'G', 'G', 'G', 'A', 'G', 'T', 'A', 'C', 'C', 'G', 'G', 'G', 'C', 'G', 'G', 'T', 'C', 'A', 'C', 'G', 'G', 'A', 'G', 'C', 'T', 'C', 'G', 'G', 'G', 'C', 'G', 'G', 'C', 'C', 'C', 'G', 'A', 'C', 'G', 'C', 'T', 'G', 'A', 'G', 'T', 'A', 'C', 'T', 'G', 'G', 'A', 'A', 'C', 'C', 'C', 'G', 'C', 'A', 'G', 'A', 'A', 'G', 'G', 'A', 'C', 'G', 'A', 'G', 'A', 'T', 'G', 'G', 'A', 'C', 'C', 'G', 'G', 'G', 'T', 'A', 'C',

In [20]:
print refseq
print h


GATTTCGTGTACCAGTTTAAGGGCGAGTGCTATTTCACCAACGGGACGGAGCGGGTGCGGCTTCTGACGAGAGACATCTATAACCGGGAGGAGCACGTGCGCTTCGACAGCGACGTGGGGGAGTACCGGGCGGTCACGGAGCTCGGGCGGCCGGACGCTGAGTACTGGAACGGGCAGAAGGAGCTCTTGGAGCGGAGGCGGGCCGAGGTGGACACGGTGTGCAGACACAACTACGGGAGGGAAGAGCTCACCACGTTGCAGCGGCGA
['G', 'A', 'T', 'T', 'T', 'C', 'G', 'T', 'G', 'T', 'AT', 'C', 'C', 'A', 'G', 'T', 'G', 'T', 'A', 'A', 'G', 'G', 'C', 'C', 'G', 'A', 'G', 'T', 'G', 'C', 'T', 'A', 'T', 'T', 'T', 'C', 'A', 'C', 'C', 'A', 'A', 'C', 'G', 'G', 'G', 'A', 'C', 'G', 'G', 'A', 'G', 'C', 'G', 'G', 'G', 'T', 'G', 'C', 'G', 'G', 'T', 'T', 'T', 'C', 'T', 'G', 'G', 'C', 'T', 'A', 'A', 'A', 'T', 'A', 'C', 'A', 'T', 'C', 'T', 'A', 'T', 'A', 'A', 'C', 'C', 'G', 'G', 'G', 'A', 'G', 'G', 'A', 'G', 'T', 'T', 'C', 'G', 'T', 'G', 'C', 'G', 'C', 'T', 'T', 'C', 'G', 'A', 'C', 'A', 'G', 'C', 'G', 'A', 'C', 'G', 'T', 'G', 'G', 'G', 'G', 'G', 'A', 'G', 'T', 'AT', 'C', 'C', 'G', 'G', 'G', 'C', 'G', 'G', 'T', 'C', 'A', 'C', 'G', 'G', 'A', 'G', 'C', 'T', 'C', 'G', 'G',

In [21]:
IUPAC={
'A':'A',
'C':'C',
'G':'G',
'T':'T',
'U':'U',
'AG':'R',
'CT':'Y',
'CG':'S',
'AT':'W',
'GT':'K',
'AC':'M',
'CGT':'B',
'AGT':'D',
'ACT':'H',
'ACG':'V',
'ACGT':'N',
'.':'.',
'-':'-'
}

In [22]:
IUPAC['AG']

'R'

In [23]:
IUPAC_1=dict([ [x[1],x[0]] for x in list(IUPAC.items())])

In [24]:
IUPAC_1['H']

'ACT'

In [25]:
s='ACGTRYSWMBHVN'
s='AHRV'
def iupac_a_seq(s):
    t=[]
    for x in s:
        t.append(IUPAC_1[x])
    return t
iupac_a_seq(s)

['A', 'ACT', 'AG', 'ACG']

In [26]:
s=['A', 'C', 'G', 'T', 'GA', 'CT', 'CG', 'AT', 'AC', 'CGT', 'ACT', 'ACG', 'ACGT']

def lista_a_iupac(s):
    ''' Recibe una lista del tipo del ejemplo, la pasa a iupac, devuelve string
    '''
    t=[]
    for x in s:
        t.append(IUPAC[''.join(sorted(list(x)))])
    return ''.join(t)
lista_a_iupac(s)

'ACGTRYSWMBHVN'

In [27]:
def seq_compare(s,t): #se supone que estan alineados y tienen la misma longitud
    match=0 
    nomatch=0 
    compared=0
    nocompared=0
    total=len(s)
    #print 'S T i M N C'
    for i, x in enumerate(s):
        y=t[i]
        if x=='-' or y=='-':
            nocompared+=1
        elif x==y: 
            match +=1
            compared +=1
        else:
            nomatch +=1
            compared +=1
        #print x, t[i], i, match, nomatch, compared
        
    return (match, nomatch, compared, nocompared, total)

def haplo_compare(s,t): #se supone que estan alineados y tienen la misma longitud
    comp=[]
    match=0 
    nomatch=0 
    compared=0
    nocompared=0
    total=len(s)
    #print 'S T i M N C'
    for i, x in enumerate(s):
        yi=set(t[i])
        xi=set(x)
        if xi==set('-') or yi==set('-'):
            nocompared+=1
            comp +='-'
        elif xi.issubset(yi): 
            match +=1
            compared +=1
            comp +='|'
        else:
            nomatch +=1
            compared +=1
            comp +='X'
        #print x, t[i], i, match, nomatch, compared
        
    return [match, nomatch, compared, nocompared, total, ''.join(comp)]
seq_compare('ASDFGH','ASEF-H')

(4, 1, 5, 1, 6)

In [28]:
seq_compare(refseq, refseq)

(267, 0, 267, 0, 267)

In [30]:
# prueba del algoritmo de alineamiento
# para adaptarse a las diferencias sin introducir gap
# -> local y no global, penalizando altamente el gap (habria que buscar un mínimo)
f=open('alineamiento.txt','w')
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
s=lista_a_seq(s)
h1=lista_a_seq(h[1])
alignments = pairwise2.align.localms(refseq,h1[:200],2,-1,-5,-5)
log(('#alignements=',len(alignments)))
for a in alignments:
    log(('--------------'))
    log((format_alignment(*a)))
    log(('a=',a))
    log(('seq_compare',seq_compare(a[0], a[1])))
#    print '--------------'
#    print format_alignment(*a)
#    print a
#    print seq_compare(a[0], a[1])

def alinear(s,t):
#    print 'align:'
#    print 's:', s
#    print 't:', t
    #log(('align:'))
    #log(('s:', s))
    #log(('t:', t))
    al = pairwise2.align.localms(s,t,2,-1,-5,-5)
#    for a in al:
 #       log(('--------------'))
  #      log((format_alignment(*a)))
   #     log(('a=',a))
    #    log(('seq_compare',seq_compare(a[0], a[1])))
    
    #log((a[0][0], a[0][1]))
    #log(('pairwise'))
    #log((a))

    return al[0][0], al[0][1]
f.close()

In [31]:
s

'ACGT[G/A][C/T][C/G][A/T][A/C][C/G][A/C][A/C][A/C]'

In [32]:
print h[1]

A


In [33]:
q='GTCATYTGDGTDGCDGGYTCDCC'
q='TTGAGBARRCCRTACATRAGC'
q='GGGCTRACRAARGCYCKRTCCC'
q='CCHTACAAYGCYGATTTYGACG'
q='GCTYATGTAYGGYYTVCTCAA'
q='GCTYTDCCKGGYATGAAYGTYCG'
r=iupac_a_seq(q)
print r

['G', 'C', 'T', 'CT', 'T', 'AGT', 'C', 'C', 'GT', 'G', 'G', 'CT', 'A', 'T', 'G', 'A', 'A', 'CT', 'G', 'T', 'CT', 'C', 'G']


In [34]:
#esto es lo que hice para Suani!!!

def gen(s,a=''):
    if len(s)>1:        #print len(s)
        for x in s[0]:
            gen(s[1:],a+x)
    else:               #es el ultimo, cerrar
        for x in s[0]:
            print a + x
s=r  
gen(s) 

GCTCTACCGGGCATGAACGTCCG
GCTCTACCGGGCATGAACGTTCG
GCTCTACCGGGCATGAATGTCCG
GCTCTACCGGGCATGAATGTTCG
GCTCTACCGGGTATGAACGTCCG
GCTCTACCGGGTATGAACGTTCG
GCTCTACCGGGTATGAATGTCCG
GCTCTACCGGGTATGAATGTTCG
GCTCTACCTGGCATGAACGTCCG
GCTCTACCTGGCATGAACGTTCG
GCTCTACCTGGCATGAATGTCCG
GCTCTACCTGGCATGAATGTTCG
GCTCTACCTGGTATGAACGTCCG
GCTCTACCTGGTATGAACGTTCG
GCTCTACCTGGTATGAATGTCCG
GCTCTACCTGGTATGAATGTTCG
GCTCTGCCGGGCATGAACGTCCG
GCTCTGCCGGGCATGAACGTTCG
GCTCTGCCGGGCATGAATGTCCG
GCTCTGCCGGGCATGAATGTTCG
GCTCTGCCGGGTATGAACGTCCG
GCTCTGCCGGGTATGAACGTTCG
GCTCTGCCGGGTATGAATGTCCG
GCTCTGCCGGGTATGAATGTTCG
GCTCTGCCTGGCATGAACGTCCG
GCTCTGCCTGGCATGAACGTTCG
GCTCTGCCTGGCATGAATGTCCG
GCTCTGCCTGGCATGAATGTTCG
GCTCTGCCTGGTATGAACGTCCG
GCTCTGCCTGGTATGAACGTTCG
GCTCTGCCTGGTATGAATGTCCG
GCTCTGCCTGGTATGAATGTTCG
GCTCTTCCGGGCATGAACGTCCG
GCTCTTCCGGGCATGAACGTTCG
GCTCTTCCGGGCATGAATGTCCG
GCTCTTCCGGGCATGAATGTTCG
GCTCTTCCGGGTATGAACGTCCG
GCTCTTCCGGGTATGAACGTTCG
GCTCTTCCGGGTATGAATGTCCG
GCTCTTCCGGGTATGAATGTTCG
GCTCTTCCTGGCATGAACGTCCG
GCTCTTCCTGGCATGA

In [35]:
hap=open('haplos.txt','w')
for h in haplos:
    hap.write(str(h[0]) + ' ' + lista_a_seq(h[1]) + '\n')
hap.close()

bak='''
for i,s in enumerate(seq):
    f=open(desc[i]+'comparacion.txt','w')
    print i,':',s
    for j,h in enumerate(haplos):
        print j,
        h1=h[1]
        s1=seq_a_lista(s)
        h1=h[1]
        log(('------------------\n')) #print '-------------------'
        x, y= alinear(lista_a_iupac(s1),lista_a_iupac(h1)) #
        res=haplo_compare(x,y)
        log((h[0]))
        log((x))
        log((str(''.join(res[-1]))))
        log((y))
        log(('match, nomatch, compared, nocompared, total, comp'))
        log((str(res[:-1])+'\n'))
    f.close()
'''

In [36]:
for i,s in enumerate(seq):
    r=open(desc[i]+'comparacion.txt','w')
    rank=[]
    print i,':',s
    for j,h in enumerate(haplos):
        print j,
        h1=h[1]
        #print 'h1:',h1
        #print lista_a_iupac(h1)
        s1=seq_a_lista(s)
        #print 's1:',s1
        #print lista_a_iupac(s1)
        #log(('------------------\n')) #print '-------------------'
        x, y= alinear(lista_a_iupac(s1),lista_a_iupac(h1)) #
        res=haplo_compare(x,y)
        res.extend([h[0],x,y])
        rank.append(res)
        debug='''
        log((h[0])) # los id
        log((x))
        log((str(''.join(res[-1]))))
        log((y))
        log(('match, nomatch, compared, nocompared, total, comp'))
        log((str(res[:-1])+'\n'))
        print((h[0])) # los id
        print((x))
        print((str(''.join(res[-1]))))
        print((y))
        print(('match, nomatch, compared, nocompared, total, comp'))
        print((str(res[:-1])+'\n'))
        '''
    #print 'sorting...'
    rank.sort(reverse=True)
    #r=open('rank.txt','w')
    for x in rank:
        r.write('Haplotipo: '+ ' '.join(x[6])+'\n')
        r.write(', '.join([str(xx) for xx in x[:5]])+'\n')
        r.write(str(x[7])+'\n')
        r.write(str(x[5])+'\n')
        r.write(str(x[8])+'\n\n')
    
    r.close()

0 : GATTTCGTGTACCAGTTTAAGTTCGAGTGCTATTTCACCAACGGGACGGGAGCGGGTGCGGCTTCTGACTAAATACATCTATAACCGGGAGGAGTTCGTGCGCTTCGACAGCGACGTGGGGGAGTACCGGGCGGTCACGGAGCTCGGGCGGCCCGACGCTGAGTACTGGAACCCGCAGAAGGACGAGATGGACCGGGTACGGGCCGAGCTGGACACGGTGTGCAGACACAACTACGGGGTGGAAGAGCTCACCACGTTGCAGCGGCGAGGT
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 20

In [37]:
rank.sort(reverse=True)
r=open('rank.txt','w')
for x in rank:
    r.write('Haplotipo: '+ ' '.join(x[6])+'\n')
    r.write(', '.join([str(xx) for xx in x[:5]])+'\n')
    r.write(str(x[7])+'\n')
    r.write(str(x[5])+'\n')
    r.write(str(x[8])+'\n\n')
    
r.close()

In [38]:
 f.close()

In [39]:
s=seq_a_lista(seq[2])
h=haplos[22]
t=h[1]
print 'S:', s
print ' '.join(s)
print 'S_iupac:'
si= ''.join(lista_a_iupac(s))
print 'si:', si
print 'haplo:',h[0]
print t
ti=seq_a_iupac(t)
print ti
ti =''.join(ti)
print 'ti:',ti

IndexError: list index out of range

In [ ]:
alignments = pairwise2.align.localms(si,ti,2,-1,-5,-10)

In [ ]:
for a in alignments:
    print '--------------'
    print format_alignment(*a)
    print a
    print seq_compare(a[0], a[1])

In [ ]:
print a[0]
print a[1]
print seq_compare(a[0],a[1])

In [ ]:
print iupac_a_seq(a[0])
print iupac_a_seq(a[1])
print haplo_compare(iupac_a_seq(a[0]),iupac_a_seq(a[1]))



In [ ]:
print haplo_compare(iupac_a_seq(a[1]),iupac_a_seq(a[0]))


In [ ]:
p= iupac_a_seq(a[0])
q= iupac_a_seq(a[1])
r= haplo_compare(iupac_a_seq(a[0]),iupac_a_seq(a[1]))
print ''.join(p)
print ''.join(q)
print ''.join(r[-1])

In [ ]:
f=open('prisec.txt')
ps=f.readlines()
f.close()
print len(ps)
a=ps[0]
b=ps[1]
print a
print b

In [ ]:
ab=unir_haplotipos(a,b)

In [ ]:
abseq=lista_a_seq(ab)
print abseq

In [ ]:
seq_a_lista(abseq)

In [ ]:
seq[0]

In [ ]:
seqs[0]